## **Diffusion Equation**

$$
\frac{d\Sigma}{dt} = \frac{3}{r}\frac{\partial}{\partial r} \left[r^{1/2} \frac{\partial}{\partial r}(\nu \Sigma r^{1/2})\right]
$$

This is evolution equation for the surface density of a geometrically thin disk

Putting this in the form of a diffusion equation:

$$
\frac{\partial f}{\partial t} = D \frac{\partial^2 f}{\partial X^2}
$$
where $X \equiv 2r^{1/2}$, $f \equiv \frac{3}{2}\Sigma X$ and $D = \frac{12\nu}{X^2}$


We need numerical values for $\nu$
$$
\nu = \alpha c_s H \\
H = \frac{c_s}{\Omega} \\
\Omega = \sqrt{\frac{G*M_s}{r^3}} \\
c_s = \sqrt{\frac{k_B * T}{\mu * mH}} \\
mH = \frac{1}{N_A} \\
\mu = 2.3 \\
$$

The Temperature profile is given by:
$$
2 \sigma_b T_{disk}^4 = \frac{9}{4}\Sigma \nu \Omega^2\\
T^3 = \frac{9}{8\sigma_b}\Sigma \frac{k_b \alpha}{\mu mH}\sqrt{\frac{GM}{r^3}}
$$

The Pressure is given by:

$$
P = \frac{c_s^2\Sigma}{H} = c_s\Sigma\Omega
$$


$$St_{max} = \frac{Q*}{\alpha c_s^2}$$


## **Pebble Accretion**

Mass Accretion is given by:
$\dot{M}_{3D} = \pi R_{acc}\rho_{p}\delta v$

Accretion Criterion:
$\tau_f = \cfrac{\xi_B \Delta v + \xi_H \Omega \hat{R}_{acc}}{GM/\hat{R}^2_{acc}}$

$\hat{R}^2_{acc} = \left(\frac{4\tau_f}{t_B}\right)^{1/2} R_B$

$R_B = \frac{GM}{\Delta v^2}$

$\xi_B = \xi_H = 0.25$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numba
import pandas as pd
import os
from dust_diffuse import *
from thermal import *
import matplotlib.cm as cm
import matplotlib.patches as ptch
from matplotlib import rc
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
import pylab

In [2]:
#Natural Constants
# T = 600 # Temperature
mu = 2.2 # mean molar mass in disk
avog = 6.02214 * 10**23 # avogadros number
mH = 1.673534 * 10**-27 # atomic hydrogen mass
kB = 1.380649 * 10**-23 #boltzmann constant
G = 6.6738 * 10**-11 #gravitational constant
Ms = 1.9886 * 10**30 #solar mass
Me = 5.972 * 10 **24 #earth mass
AU = 1.4959787 * 10**11 #astronomical units
yrs2sec = 3.1536 * 10**7 #convert years to seconds
sb = 5.6704*10**-8

In [3]:
#Disk Parameters
alpha1 = 1e-2
alpha2 = 1.0 * 10**-3
alpha3 = 1.0 * 10**-4
rin = 0.05 * AU #inner radius of the disk
rout = 30 * AU #outer radius of the disk
sigma_in  = 2 * 10**5 #boundary condition at t=0
sigma_max = sigma_in*2 #boundary condition at t=final_time
sigma_min = 1 * 10**2
distance = rout - rin #distance from inner radius to outer radius

In [4]:
#Temporal discretization
max_years = 100
dyr = .1
dt = dyr * yrs2sec #timestep
final_time = max_years*yrs2sec + 1 #total diffusion time in seconds
t_save_interval = 100 #every ten years
t = np.arange(0, final_time, dt)

In [5]:
#Spacial discretization

n = 300 #number of steps / number of points to plot
dr = distance/n #distance between each spacial step
cont = True #Boolean for continuous alpha distribution

In [6]:
#Other Parameters

# St = .01 #Stokes Number
Q = 1
Tcor = 1800 #10% corundum
Tcalc = 1700 #10% calcium
Tfor = 1400 #80% forsterite

cor_ratio = .1
calc_ratio = .1
for_ratio = .8

phase_change = 1400

In [7]:
#Initialize grid spaces

dist = np.empty(n)
Omega = np.empty(n)
X = np.empty(n)

In [8]:
#Initialize Staggered Grid

dist_stag = np.empty(n-1)
Omega_stag = np.empty(n-1)

In [9]:
#Boundary Conditions

f_in = 0
f_out = 0

In [10]:
@numba.njit
def make_alpha_grid(cont):
    if(cont):
        alphas = np.linspace(alpha1, alpha3, n)
    else:
        alphas = np.empty(n)
        for i in range(n):
            if i < (n//3):
                alphas[i] = alpha1
            elif i < (2*n//3):
                alphas[i] = alpha2
            else:
                alphas[i] = alpha3
    return alphas

In [11]:
@numba.njit
def update_alpha(alphas, T):
    for idx, temp in enumerate(T):
        if(temp >= phase_change):
            alphas[idx] = alpha2 + 9e-3*min((temp-phase_change)/100, 1.)
        else:
            alphas[idx] = alpha2
    return alphas

In [12]:
def calc_dist_omega_stag():
    for i in range(1, n):
        dist_stag[i-1] = 0.5 * (dist[i] + dist[i-1])
        Omega_stag[i-1] = np.sqrt(G * Ms / dist_stag[i-1] ** 3)

In [13]:
def update_stag_grid(alphas, cs2):
    cs2_stag = np.array([0.5 * (cs2[i] + cs2[i-1]) for i in range(1, n)])
    alphas_stag = np.array([0.5 * (alphas[i] + alphas[i-1]) for i in range(1, n)])
    return (alphas_stag, cs2_stag)

In [14]:
@numba.njit
def update_nu(cs2, alphas):
    return alphas*cs2/Omega

In [15]:
@numba.njit
def update_St(alpha_stag):
    return Q/(cs2_stag*alpha_stag)

In [16]:
@numba.njit
def update_D(nu):
    return 12 * nu / (X ** 2)

In [17]:
def calc_init_params():
    """
    Calculate the initial parameters values for time t = 0.
    """
    sigma_gas = np.empty(n)
    sigma_dust = np.empty((3,n))
    sigma_pebble = np.zeros((3,n))
    sigma_evap_init = np.zeros((3,n))
    alphas = make_alpha_grid(cont)
    for i in range(n):
        dist[i] = (rin + (rout-rin)*i/(n-1))
        sigma_gas[i] = sigma_in * AU / dist[i]
        if (sigma_gas[i]>sigma_max):
            sigma_gas[i] = sigma_max
        if (dist[i]/AU > 15):
            sigma_gas[i] = sigma_min
        sigma_dust[0,i] = sigma_gas[i] * 0.005 * cor_ratio
        sigma_dust[1,i] = sigma_gas[i] * 0.005 * calc_ratio
        sigma_dust[2,i] = sigma_gas[i] * 0.005 * for_ratio
        X[i] = 2 * np.sqrt(dist[i])
        Omega[i] = np.sqrt(G * Ms / (dist[i] ** 3))
    cs2, T, P, sigma_dust, sigma_evap, sigma_pebble = calc_thermal_struc(sigma_gas, sigma_dust, sigma_pebble, sigma_evap_init, alphas, Omega)
    calc_dist_omega_stag()
    alphas = update_alpha(alphas, T)
    alphas_stag, cs2_stag = update_stag_grid(alphas, cs2)
    St = Q/(alphas_stag * cs2_stag)
    nu = alphas*cs2/Omega
    v_gas = calc_gas_vel(nu, sigma_gas, dist, dist_stag)
    v_dust = calc_dust_vel(St, cs2_stag, v_gas, P)
    v_pebble = np.zeros(n)
    D = 12 * nu / (X ** 2)
    f = (1.5 * X * sigma_gas)
    return (sigma_dust, sigma_gas, sigma_evap, sigma_pebble, v_dust, v_gas, v_pebble, cs2, nu, D, f, alphas, alphas_stag, cs2_stag, St)

In [18]:
def calc_dust_vel(St, cs2_stag, v_gas, press):
    press_stag = [0.5*(press[i]+press[i-1]) for i in range(1,n)]
    dPdr = [(press[i] - press[i-1])/dist_stag[i-1] for i in range(1,n)]
    rho_stag = press_stag/cs2_stag
    v_dust = (St/(1+(St)**2)) * (1 / (rho_stag * Omega_stag))*dPdr
    v_dust = np.append(v_dust, 0)
#     v_dust = v_dust + v_gas
    return v_dust

In [19]:
# @numba.njit
def calc_pebble_vel(v_dust):
#     Stoke = 0.05
#     press_stag = [0.5*(press[i]+press[i-1]) for i in range(1,n)]
#     dPdr = [(press[i] - press[i-1])/dist_stag[i-1] for i in range(1,n)]
#     rho_stag = press_stag/cs2_stag
#     v_dust = (Stoke/(1+(Stoke)**2)) * (1 / (rho_stag * Omega_stag))*dPdr
#     v_dust = np.append(v_dust, 0)
#     v_dust = v_dust + v_gas
    return v_dust * 10

In [20]:
@numba.njit
def calc_pebble_growth(sigma_pebble, sigma_dust):
    tau = 1.988e-7 * 1e4*yrs2sec / Omega
    sigma_pebble += sigma_dust * dt / tau
    sigma_dust -= sigma_dust * dt / tau
    return (sigma_pebble, sigma_dust)

In [21]:
@numba.njit
def calc_gas_evol(f, dt, D):
    """Outputs the surface density at a specific dt."""
    df_dt = np.empty(n)
    for j in range(1, n-1):
        dX1 = X[j] - X[j-1]
        dX2 = X[j+1] - X[j]
        D1 = 0.5 * (D[j] + D[j-1])
        D2 = 0.5 * (D[j+1] + D[j])
        df_dt[j] = D1 * ((-(f[j] - f[j-1])/dX1**2)) + D2 * ((f[j+1]-f[j])/dX2**2)
    dX_final = X[-1]-X[-2]
    dX_in = X[1]-X[0]
    df_dt[0] = D[0] * (-(f[0] - f_in)/dX_in**2 + (f[1]-f[0])/dX_in**2)
    df_dt[n-1] = D[n-1] * (-(f[n-1] - f[n-2])/dX_final**2 + (f_out-f[n-1])/dX_final**2)
    f_new = f + df_dt * dt
    sigma_at_time = [2*f_new[k]/(3*X[k]) for k in range(n)]
    return (f_new, sigma_at_time)

In [22]:
# @numba.njit
def calc_disk_mass(sigma):
    """Calculate the mass at a specific dt"""
    disk_mass = 0
    for j in range(n):
        disk_mass += 2 * np.pi * dist[j] * dr * sigma_gas[j]
        disk_mass += 2 * np.pi * dist[j] * dr * sigma_dust[j]
#     print(f'The mass of the disk is {disk_mass}.')
    return disk_mass

In [23]:
def save2dir(**alpha_run):
    if(alpha_run):
        if(alpha_run == alpha1):
            output_dir = 'output_a1/'
            filename = 'output_a1/disk_'
        elif(alpha_run == alpha2):
            output_dir = 'output_a2/'
            filename = 'output_a2/disk_'
        else:
            output_dir = 'output_a3/'
            filename = 'output_a3/disk_'
    else:
        output_dir = 'output/'
        filename = 'output/disk_'
    return (output_dir, filename)

In [24]:
#Time Evolution Main

sigma_dust, sigma_gas, sigma_evap, sigma_pebble, v_dust, v_gas, v_pebble, cs2, nu, D, f, alphas, alphas_stag, cs2_stag, St = calc_init_params()
output_dir, filename = save2dir()

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
for i in range(len(t)):
    f_new, sigma_gas = calc_gas_evol(f, dt, D)
    sigma_pebble, sigma_dust = calc_pebble_growth(sigma_pebble, sigma_dust)
    for el in range(3):
        sigma_dust[el,:] = calc_evol(sigma_gas, sigma_dust[el,:], nu, v_dust, dist, dt)
        sigma_pebble[el,:] = calc_evol(sigma_gas, sigma_pebble[el,:], nu, v_pebble, dist, dt)
        sigma_evap[el,:] = calc_evol(sigma_gas, sigma_evap[el,:], nu, v_gas, dist, dt)
    cs2, T, P, sigma_dust, sigma_evap, sigma_pebble = calc_thermal_struc(sigma_gas, sigma_dust, sigma_pebble, sigma_evap, alphas, Omega)
    alphas = update_alpha(alphas, T)
#     print(np.transpose(np.vstack([np.sum(sigma_dust), np.sum(sigma_pebble), T[0]])))
    alphas_stag, cs2_stag = update_stag_grid(alphas, cs2)
    nu = update_nu(cs2, alphas)
    D = update_D(nu)
    St = update_St(alphas_stag)
    v_dust = calc_dust_vel(St, cs2_stag, v_gas, P)
    v_pebble = calc_pebble_vel(v_dust)
    v_gas = calc_gas_vel(nu, sigma_gas, dist, dist_stag)
    f = f_new
    if (i%(t_save_interval/dyr) == 0):
        output = np.transpose(np.vstack([dist/AU, sigma_gas, sigma_dust, sigma_pebble, sigma_evap, T]))
        np.savetxt(filename + str(int(i*dyr)) + '.txt', output, delimiter=',', newline='\n')# save results to file

10.314584036713114
10.317048720894086


/Users/Rohan/opt/anaconda3/lib/python3.8/site-packages/numba/core/ir_utils.py:2119: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'sigma_gas' of function 'calc_thermal_struc'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "thermal.py", line 98:
@numba.njit
def calc_thermal_struc(sigma_gas, sigma_dust, sigma_pebble, sigma_evap, alphas, Omega):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


10.314967481590086
10.314922857784337
10.31722252725074
10.31474872114224
10.314704098282876
10.317004621061487
10.314531248825054
10.3144866269065
10.316962967263116
10.314343574314378
10.314298953207729
10.316775263919515
10.314156883514936
10.314112263215934
10.316587311127494
10.313970855647213
10.313926236152989
10.316400008268428
10.313785010142995
10.313740391452761
10.316212879407788
10.313599049012474
10.31355443112673
10.316025648947967
10.31341283354521
10.313368216465056
10.315838179269484
10.313226259735178
10.313181643462165
10.315826875145694
10.313073759055014


In [25]:
# i = 30
# sigma_dust[:,i], sigma_evap[:,i], T[i], kpa[i] = calc_middiskT(sigma_gas[i], sigma_dust[:,i], sigma_evap[:,i], alphas[i], Omega[i])

In [26]:
def plot():
    %matplotlib
    t_plot = [100]
    for time in t_plot:
        df = pd.read_csv('output/disk_' + str(int(t_plot[-1])) + '.txt', delimiter=',', header=None)
        sigma_gas_plot = df[1].to_numpy().astype(np.float)
        cor_dust = df[2].to_numpy().astype(np.float)
        calc_dust = df[3].to_numpy().astype(np.float)
        for_dust = df[4].to_numpy().astype(np.float)
        cor_peb = df[5].to_numpy().astype(np.float)
        calc_peb = df[6].to_numpy().astype(np.float)
        for_peb = df[7].to_numpy().astype(np.float)
        sigma_dust_plot = cor_dust+calc_dust+for_dust
        sigma_peb_plot = cor_peb+calc_peb+for_peb
        print(calc_peb)
        temp_plot = df[11].to_numpy().astype(np.float)
        dg_ratio = sigma_dust_plot/sigma_gas_plot
        fig, axes = plt.subplots(2,2)
        ax1 = axes[0][0]
        ax2 = axes[1][0]
        ax3 = axes[0][1]
        ax4 = axes[1][1]
#         cm = pylab.get_cmap('jet') #gist_rainbow
#         plt.rcParams['font.family']='sans-serif'
#         plt.rcParams['axes.linewidth']= 1.0
#         plt.rcParams['font.size']     = 14
#         plt.rcParams['figure.figsize']= 4*1.414, 4
#         rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
#         rc('text',usetex=False)
#         rc('text.latex', preamble=r'\usepackage{sfmath}')
        ax1.set_title(f'{int(time)} years')
        ax1.set_xlabel('Distance (AU)')
        ax1.set_ylabel('Dust')
        ax1.set_yscale('log')
        ax1.tick_params(direction='in', length=3,   which='major', top=True, right=True)
        ax1.tick_params(direction='in', length=1.5, which='minor', top=True, right=True)
        ax1.plot(dist/AU, cor_dust, label='Aluminum', color='green')
        ax1.plot(dist/AU, calc_dust, label='Calcium', color='red')
        ax1.plot(dist/AU, for_dust, label='Magnesium', color='blue')
        ax1.legend(fontsize='x-small')
        ax2.tick_params(direction='in', length=3,   which='major', top=True, right=True)
        ax2.tick_params(direction='in', length=1.5, which='minor', top=True, right=True)
        ax2.set_xlabel('Distance (AU)')
        ax2.set_ylabel('Temperature')
        ax2.set_yscale('log')
        ax2.plot(dist/AU, temp_plot)
        ax3.tick_params(direction='in', length=3,   which='major', top=True, right=True)
        ax3.tick_params(direction='in', length=1.5, which='minor', top=True, right=True)
        ax3.set_xlabel('Distance (AU)')
        ax3.set_ylabel('Dust/Gas')
        ax3.set_yscale('log')
        ax3.plot(dist/AU, dg_ratio)
        ax4.tick_params(direction='in', length=3,   which='major', top=True, right=True)
        ax4.tick_params(direction='in', length=1.5, which='minor', top=True, right=True)
        ax4.set_xlabel('Distance (AU)')
        ax4.set_ylabel('Evap')
        ax4.set_yscale('log')
#         ax4.set_ylim(1e-8,1e1)
        ax4.plot(dist/AU, cor_peb, label='Aluminum', color='green')
        ax4.plot(dist/AU, calc_peb, label='Calcium', color='red')
        ax4.plot(dist/AU, for_peb, label='Magnesium', color='blue')
        ax4.legend(fontsize='x-small')
    plt.show()
    plt.tight_layout()
    plt.savefig(f'evol_{int(t_plot[-1])}.pdf', transparent=True)

In [27]:
# files = ['tau_1e5_St-5e-2', 'tau-1e2', 'tau-1e4_St-5e-2','tau-1e4_St-x10']
# compare_temp(files)
plot()

Using matplotlib backend: MacOSX
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.76171861e-07
 1.11658319e-06 2.46517211e-06 4.26124410e-06 6.5528

In [28]:
# def compare_temp(filenames):
#     for file in filenames:
#         df = pd.read_csv(file + '/disk_' + str(int(t_plot[-1])) + '.txt', delimiter=',', header=None)
#         temp_plot = df[11].to_numpy().astype(np.float)
#         cor_peb = df[5].to_numpy().astype(np.float)
#         calc_peb = df[6].to_numpy().astype(np.float)
#         for_peb = df[7].to_numpy().astype(np.float)
# #         plt.figure(idx+1)
#         plt.title(f'Pebble Evolution at t={int(t_plot[-1])} years')
#         plt.xlabel('Distance (AU)')
#         plt.ylabel('SD')
#         plt.yscale('log')
#         plt.plot(dist/AU, cor_peb, label='cor', color='green')
#         plt.plot(dist/AU, calc_peb, label='calc', color='red')
#         plt.plot(dist/AU, for_peb, label='for', color='blue')
#         plt.legend()
#         plt.savefig("pebble_comparison.pdf")
#         plt.show()

In [29]:
# print(f'The range of Stokes Numbers is {min(St)} to {max(St)}')

In [30]:
# ax.tick_params(direction='in', length=3,   which='major', top=True, right=True)
# ax.tick_params(direction='in', length=1.5, which='minor', top=True, right=True)

# ax.set_xlim([1e-4, 1e-1])
# ax.set_ylim([1e-6, 1e-1])


# plt.tight_layout()
# plt.savefig("fig_ratio.pdf", transparent=True)

In [31]:
# import matplotlib.cm      as cm
# import matplotlib.pyplot  as plt
# import matplotlib.patches as ptch
# from   matplotlib import rc
# from   matplotlib.font_manager import FontProperties
# from   matplotlib.ticker       import MultipleLocator, FormatStrFormatter
# import pylab
# cm = pylab.get_cmap('jet') #gist_rainbow

# plt.rcParams['font.family']='sans-serif'
# plt.rcParams['axes.linewidth']= 1.0
# plt.rcParams['font.size']     = 14
# plt.rcParams['figure.figsize']= 4*1.414, 4
# rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
# rc('text',usetex=True)
# rc('text.latex', preamble=r'\usepackage{sfmath}')

In [32]:
import numpy as np

def test(arr):
    return arr +33

In [33]:
a = np.zeros([2,2])
for i in range(1):
    a[i,:] = test(a[i,:])
a

array([[33., 33.],
       [ 0.,  0.]])